In [ ]:
import os
import pickle
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import plot_tag_data_interactive, plot_tag_data, load_color_mapping, save_color_mapping, plot_depth_correction
from pyologger.calibrate_data.calibrate_acc_mag import *
from pyologger.process_data.feature_generation_utils import get_heart_rate

# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
?get_heart_rate

In [ ]:
from plotnine import ggplot, aes, geom_histogram, labs

ecg_sampling_rate = int(data_pkl.info['UF-01']['datetime_metadata']['fs'])
hr = get_heart_rate(data_pkl.data['UF-01']['ecg'], 
                    fs= ecg_sampling_rate, 
                    search_radius=60, 
                    filter_threshold=120,
                    algo='heartpy',
                    correct_peaks=True)
data_pkl.data['UF-01']['hr'] = hr


print(hr)

# Assuming 'hr' is a pandas Series or a column in a DataFrame
# For demonstration, let's assume it's part of a DataFrame named df
df = pd.DataFrame({'hr': hr})

# Create the histogram
histogram_plot = (
    ggplot(df, aes(x='hr')) +
    geom_histogram(binwidth=5, fill='lightblue', color='white', alpha=0.7) +
    labs(title='Histogram of Heart Rate (HR)', x='Heart Rate (HR)', y='Count')
)

# To display the plot
print(histogram_plot)

imu_channels_to_plot = ['depth', 'accX', 'accY', 'accZ', 'odba']
ephys_channels_to_plot = ['ecg', 'hr']
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'exhalation_breath': 'depth',
    'heartbeat_manual_ok': 'ecg',
}

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, imu_sampling_rate=1, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, note_annotations= notes_to_plot,
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)